In [8]:
# web scraping
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By

# generate the csv
import pandas as pd

Setting up Selenium (linux)

In [9]:
# path
CHROMEDRIVER_PATH = '/usr/local/bin/chromedriver'
s = Service(CHROMEDRIVER_PATH)
WINDOW_SIZE = "1920,1080"

# options
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--window-size=%s" % WINDOW_SIZE)
chrome_options.add_argument('--no-sandbox')
driver = webdriver.Chrome(service=s, options=chrome_options)

Incompatible release of chromedriver (version 111.0.5563.64) detected in PATH: /bin/chromedriver


Navigation

In [10]:
char_list = []

# go to website
driver.get("https://www.litcharts.com/lit/burial-rites/characters")

char_elems = driver.find_elements(By.CSS_SELECTOR, 'div.name, h2.naked.name.index-list-item-title')

# iterate through the tags and print their text content
for elem in char_elems:
    char_list.append({'character': elem.text})

# close the browser
driver.close()

Load into CSV data file

In [11]:

import re 
df = pd.DataFrame(char_list)

# Accounts for characters with an alias in brackets (alias column)
df['alias'] = df['character'].apply(lambda x: re.findall(r'\((.*?)\)', x))
df['alias'] = df['alias'].apply(lambda x: x[0] if len(x) > 0 else '')

# Remove brackets and text within brackets (characters column)
df['character'] = df['character'].apply(lambda x: re.sub("[\(].*?[\)]", "", x))


# Any entity with 'reverend', 'uncle', 'aunty', 'and', is given the first name of what comes after
# Any entity without returns the first word of its string
pattern = r'(?:reverend|uncle|aunty|and)\s+(\w+)'

def filter_firstname(s):
    match = re.search(pattern, s, re.IGNORECASE)
    if match:
        return match.group(1)
    else:
        return s.split(' ', 1)[0]

# Apply the filter (character_firstname column)
df['character_firstname'] = df['character'].apply(filter_firstname)

pd.set_option('display.max_rows', None)
df

df.to_csv('characters.csv', index=False, header=True)